In [1]:
%connect_info

{
  "shell_port": 58390,
  "iopub_port": 58391,
  "stdin_port": 58392,
  "control_port": 58394,
  "hb_port": 58393,
  "ip": "127.0.0.1",
  "key": "4f187bef-c229bc1324cee640afed4f3f",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-3231c062-69b6-4847-818a-66255943b22c.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
%matplotlib inline

In [3]:
import sys
sys.argv = sys.argv[:1]

In [5]:
import os
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from utils import build_model_name, convert_flags_to_dict, define_cnn_flags

from transformers import AutoTokenizer, TFAutoModel


SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

ImportError: dlopen(/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/_C.cpython-37m-darwin.so, 2): Library not loaded: @rpath/libc++.1.dylib
  Referenced from: /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/torch/_C.cpython-37m-darwin.so
  Reason: image not found

In [ ]:
bert_type = 'distilbert-base-multilingual-cased'

In [137]:
with open('{}/data/{}.tokenized.pkl'.format(cwd, bert_type),
              'rb') as f:
        data = pickle.load(f)
print(data.keys())

dict_keys(['DE', 'GA', 'HI', 'PT', 'ZH'])


In [111]:
ga = data['PT']

In [133]:

x_train, y_train = [], []

for code in data.keys():

    true_x, true_y = [], []
    false_x, false_y = [], []
    for xsample, ysample in zip(data[code]['x_train'], data[code]['y_train']):
        if 1 in ysample:
            true_x.append(xsample)
            true_y.append(ysample)
    
    
    max_len = max([len(y) for y in true_y])

    for xsample, ysample in zip(data[code]['x_train'], data[code]['y_train']):
        if 1 not in ysample and len(ysample) < max_len:
            false_x.append(xsample)
            false_y.append(ysample)
    false_x = np.array(false_x)
    false_y = np.array(false_y)
    
    np.random.seed(SEED)
    idx = np.random.randint(len(false_y), size=len(true_y))
    false_x = false_x[idx].tolist()
    false_y = false_y[idx].tolist()
    
    x_train += true_x + false_x
    y_train += true_y + false_y

In [134]:
len(x_train)

26564

In [120]:
idx = np.random.randint(len(false_y), size=len(true_y))
idx

array([2777, 3378,  392, ..., 2413, 1238, 1486])

In [114]:
false_x = false_x[idx].tolist()
false_y = false_y[idx].tolist()

In [124]:
len(false_y)

37

In [116]:
len(false_x + false_y) == len(true_x + true_y)

True

In [129]:
lens = [len(x) for code in data.keys() for x in data[code]['x_train']]
len(lens)

66338

66338

In [144]:
x_train, y_train = [], []
x_dev, y_dev = [], []
for code in data.keys():

    true_x, true_y = [], []
    false_x, false_y = [], []
    for xsample, ysample in zip(data[code]['x_train'], data[code]['y_train']):
        if 1 in ysample:
            true_x.append(xsample)
            true_y.append(ysample)


    max_len = max([len(y) for y in true_y])
    print(max_len)

    for xsample, ysample in zip(data[code]['x_train'], data[code]['y_train']):
        if 1 not in ysample and len(ysample) < max_len:
            false_x.append(xsample)
            false_y.append(ysample)
    false_x = np.array(false_x)
    false_y = np.array(false_y)

    np.random.seed(SEED)
    idx = np.random.randint(len(false_y), size=len(true_y))
    false_x = false_x[idx].tolist()
    false_y = false_y[idx].tolist()

    x_train += true_x + false_x
    y_train += true_y + false_y

    x_dev += data[code]["x_dev"]
    y_dev += data[code]["y_dev"]

# del data

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train)
max_len = x_train.shape[1]
y_train = tf.keras.preprocessing.sequence.pad_sequences(y_train, maxlen=max_len)

x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                    y_train,
                                                    test_size=0.15,
                                                    random_state=SEED)

print(x_train.shape, x_train.shape, y_train.shape)
print(x_val.shape, x_val.shape, y_val.shape)
x_train = [x_train, (x_train > 0).astype(int)]
x_val = [x_val, (x_val > 0).astype(int)]


x_dev = tf.keras.preprocessing.sequence.pad_sequences(x_dev, maxlen=max_len)
x_dev = [x_dev, (x_dev > 0).astype(int)]

seq_lens = [len(seq) for seq in y_dev]
y_dev = tf.keras.preprocessing.sequence.pad_sequences(y_dev, maxlen=max_len)
print(x_dev[0].shape, x_dev[1].shape, y_dev.shape)

175
228
99
338
204
(22579, 338) (22579, 338) (22579, 338)
(3985, 338) (3985, 338) (3985, 338)
(4330, 338) (4330, 338) (4330, 338)


In [141]:
type(data['GA']['x_dev'])

list